In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras import backend as K
from sklearn.ensemble import RandomForestRegressor

Using TensorFlow backend.


In [3]:
ammount_windows = 4
model_w1 = 0
model_w2 = 0
model_w3 = 0
model_w4 = 0
windows = [model_w1, model_w2, model_w3, model_w4]
uses = 'all'
windows_files = ['data/model/model_' +uses+ '_w1.pkl', 'data/model/model_' +uses+ '_w2.pkl', 'data/model/model_' +uses+ '_w3.pkl', 'data/model/model_' +uses+ '_w4.pkl']
target = pd.read_csv('data/target_competencia_ids.csv')

for i in range(ammount_windows):
    windows[i] = pd.read_pickle(windows_files[i])

    #Completo NaN's
features = {'n_auctions':0, 'diff_auctions': 'mean', 'prom_time_auction': 'mean', 'first_auction_sec': 'mean',
            'last_auction_sec': 'mean', 'ref_type_id_1': 0, 'ref_type_id_7': 0, 'source_id_0': 0, 'source_id_1': 0, 'source_id_2':0,
            'source_id_3':0, 'source_id_4':0, 'source_id_5':0, 'source_id_6':0,
            'source_id_7':0, 'source_id_8':0, 'source_id_9':0, 'mean_day': 'mean', 'day_0': 0,'day_1': 0, 'day_2': 0,
            'n_events': 0, 'mean_time_events': 'mean', 'wifi_events_mean': 'mean', 'diff_events': 0, 'first_event_sec': 0,
           'last_event_sec': 0, 'attributed_events_mean': 0, 'touchX_mean': 'mean', 'touchY_mean': 'mean', 'timeToClick_mean': 'mean',
           'latitude_mean': 'mean', 'longitude_mean': 'mean', 'n_clicks': 0, 'mean_time_install': 0, 'timeToClick_mean': 'mean',
           'first_install_sec': 0, 'last_install_sec': 0, 'n_installs': 0, 'mean_time_install': 'mean', 'attributed_installs_mean': 'mean',
           'wifi_installs_mean': 'mean', 'diff_installs': 'mean', 'mean_time_install': 'mean', 'wifi_connection': 'mean', 'first_click_sec': 'mean',
           'last_click_sec': 'mean', 'diff_clicks': 'mean', 'prom_time_click': 'mean', 'mean_time_auction': 'mean', 'mean_time_click': 'mean',
           'kind_Add_to_cart': 0, 'kind_App First Start': 0, 'kind_EVENT_OPEN_APP': 0, 'kind_MobileSignup': 0, 'kind_Open': 0, 'kind_Other': 0,
            'kind_Pre_checkout': 0, 'kind_Registration': 0, 'kind_Sign In': 0, 'kind_account_summary_first_step': 0, 'kind_af_app_opened': 0,
            'kind_af_content_view': 0, 'kind_af_view_product': 0, 'kind_app_open': 0, 'kind_app_opened': 0, 'kind_list view': 0, 'kind_login_success': 0,
            'kind_pax_app_open': 0, 'kind_product view': 0, 'kind_reattribution': 0, 'kind_sessionbegin': 0, 'source_0': 0, 'source_1': 0, 'source_2': 0, 
            'source_3': 0, 'source_4': 0, 'source_5': 0, 'source_6': 0, 'source_7': 0, 'source_8': 0, 'source_9': 0, 'kind_startsessionplayback': 0}
for i in range(ammount_windows):
    for feature, method in features.items():
        if feature not in windows[i]:
            continue
        if method == 0:
            windows[i][feature] = windows[i][feature].fillna(0)
        elif method == 'mean':
            windows[i][feature] = windows[i][feature].fillna(windows[i][feature].mean())
        elif method == 'max':
            windows[i][feature] = windows[i][feature].fillna(windows[i][feature].max())
        elif method == 'min':
            windows[i][feature] = windows[i][feature].fillna(windows[i][feature].min())
        elif method == -1:
            windows[i][feature] = windows[i][feature].fillna(-1)
            
model = windows[0]
for i in range(1, ammount_windows):
    model = pd.concat([model, windows[i]], ignore_index=True, sort=False)
    
model.fillna(0, inplace=True)

In [5]:
windows[0].columns

Index(['ref_hash', 'n_auctions', 'diff_auctions', 'mean_time_auction',
       'first_auction_sec', 'last_auction_sec', 'last_auction_sec_to_end',
       'ref_type_id_1', 'ref_type_id_7', 'source_0',
       ...
       'source_id_7', 'source_id_8', 'touchX_mean', 'touchY_mean',
       'touchs_in_bottom', 'touchs_in_bottom2', 'latitude_mean',
       'longitude_mean', 'time_appearence', 'time_appearence_install'],
      dtype='object', length=165)

In [20]:
model.isna().sum()

ref_hash                                    0
n_auctions                                  0
diff_auctions                               0
mean_time_auction                           0
first_auction_sec                           0
last_auction_sec                            0
last_auction_sec_to_end                     0
ref_type_id_1                               0
ref_type_id_7                               0
source_0                                    0
source_1                                    0
source_2                                    0
source_3                                    0
source_4                                    0
source_5                                    0
source_6                                    0
source_7                                    0
source_8                                    0
source_9                                    0
day_0                                       0
day_1                                       0
day_2                             

In [3]:
model.shape

(1286041, 178)

**Modelos**

In [4]:
target_features = ['time_appearence', 'time_appearence_install']
objective = 1

model.drop(columns='ref_hash', inplace=True)
max_seconds = 87000
train, test = train_test_split(model, test_size=0.2)
train_Y = train[target_features[objective]]
train_X = train.drop(columns=target_features)
test_Y = test[target_features[objective]]
test_X = test.drop(columns=target_features)

**Base (promedio)**

In [7]:
mean_time = train_Y.mean()
np.sqrt(mean_squared_error(train_Y, np.full(train_Y.shape, mean_time)))

105554.37511883573

In [8]:
np.sqrt(mean_squared_error(test_Y, np.full(test_Y.shape, mean_time)))

105571.09085473178

**Random Forest**

In [41]:
random_f = RandomForestRegressor(max_depth=10, random_state=1, n_estimators=100)
random_f.fit(train_X, train_Y)

KeyboardInterrupt: 

In [10]:
np.sqrt(mean_squared_error(test_Y, random_f.predict(test_X)))

92333.71456233726

**Lasso**

In [105]:
lasso = LassoCV(max_iter=1e7, random_state=1, cv=5)
lasso.fit(train_X, train_Y)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=10000000.0, n_alphas=100, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=1, selection='cyclic',
    tol=0.0001, verbose=False)

In [107]:
np.sqrt(mean_squared_error(test_Y, lasso.predict(test_X)))

75807.73438358566

**Red Neuronal**

In [66]:
nn = Sequential([
    BatchNormalization(input_shape = (67, )),
    Dense(70),
    Activation('relu'),
    Dense(30),
    Activation('relu'),
    Dense(10),
    Dense(1)
])

In [68]:
nn.compile(optimizer='rmsprop',
              loss='mse')
nn.fit(train_X, train_Y, epochs=5, batch_size=32)

Epoch 1/5
254660/254660 [==============================] - 10s 40us/step - loss: 9079748097.1420
Epoch 2/5
254660/254660 [==============================] - 9s 36us/step - loss: 9065587434.1617
Epoch 3/5
254660/254660 [==============================] - 9s 36us/step - loss: 9053894049.5215
Epoch 4/5
254660/254660 [==============================] - 11s 44us/step - loss: 9047443615.3943
Epoch 5/5
254660/254660 [==============================] - 11s 45us/step - loss: 9040465155.1163


In [69]:
nn_pred_train = nn.predict(train_X)
nn_pred_test = nn.predict(test_X)

In [70]:
np.sqrt(mean_squared_error(train_Y, nn_pred_train))

94794.28430887965

In [71]:
np.sqrt(mean_squared_error(test_Y, nn_pred_test))

94565.22412627791

**XGBoost**

In [14]:
%time
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.5,  subsample=0.8,
                max_depth = 5, alpha = 10, n_estimators = 500, gamma=10, random_state=45)
xg_reg.fit(train_X, train_Y)

Wall time: 0 ns


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=10, learning_rate=0.5, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=45,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [15]:
xg_pred_train = xg_reg.predict(train_X)
xg_pred_test = xg_reg.predict(test_X)

In [16]:
np.sqrt(mean_squared_error(train_Y, xg_pred_train))

91985.50903994139

In [17]:
np.sqrt(mean_squared_error(test_Y, xg_pred_test))

94830.30127373632

In [142]:
xg_pred_test

array([252201.64, 254271.58, 253163.  , ..., 256294.08, 253890.25,
       249540.72], dtype=float32)

In [18]:
xg_reg.save_model('modelos/xgboost_all_auctions')

In [16]:
space  = [Integer(80, 100, name='n_estimators'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(8, 15, name='max_depth'),
          Real(0.01, 0.5, "log-uniform", name='alpha'),
          Real(0.05, 1, name='colsample_bytree')]

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set scikit-learn
# estimator parameters
@use_named_args(space)
def objective(**params):
    xg_reg.set_params(**params)

    return -np.mean(cross_val_score(xg_reg, train_X, train_Y, cv=5, n_jobs=-1,
                                    scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)

KeyboardInterrupt: 

In [98]:
res_gp

          fun: 46791.59921416599
    func_vals: array([53302.18774591, 55908.08816728, 49897.50923364, 54500.26530089,
       52474.47354897, 47802.56045926, 51507.2941305 , 56755.03832791,
       54297.66881425, 54950.68425361, 46959.18236723, 57236.31920252,
       46955.38511696, 46959.5869603 , 46957.21574916, 46957.21574916,
       52937.21386543, 46955.47322116, 46952.75822945, 46950.40576502,
       46948.36594847, 46952.19743406, 46952.96500169, 46953.01394625,
       46953.10311067, 46953.92190269, 46953.88803166, 46953.06934106,
       46953.86868935, 46952.96303067, 46952.96353762, 46951.89323727,
       46951.88656024, 46952.14221605, 47052.63103154, 47771.65010067,
       46805.32557035, 46888.49285122, 47274.04960373, 46813.0681015 ,
       48935.72359859, 46811.9639108 , 46888.45301444, 48344.67448439,
       47274.08474032, 46797.39209552, 46886.67221753, 46801.18192607,
       46791.59921417, 46948.50109856])
       models: [GaussianProcessRegressor(alpha=1e-10, copy_X

In [7]:
train_X = 0
test_X = 0

**lightgbm**

In [5]:
lgb_train = lgb.Dataset(train_X, train_Y)
lgb_eval = lgb.Dataset(test_X, test_Y, reference=lgb_train, silent=True)

In [14]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2_root',
    'num_leaves': 150,
    'max_bin': 200,
    'learning_rate': 0.02,
    'colsample_bytree': 1,
    'verbose': 0,
    'bagging_fraction':0.5,
    'max_depth':15
}

gbm_model = lgb.train(params,
                lgb_train,
                num_boost_round=1600,
                valid_sets=lgb_eval,
                early_stopping_rounds=20,
                verbose_eval=100)

gbm_pred_train = gbm_model.predict(train_X, num_iteration=gbm_model.best_iteration)
gbm_pred_test = gbm_model.predict(test_X, num_iteration=gbm_model.best_iteration)

Training until validation scores don't improve for 20 rounds.
[100]	valid_0's rmse: 77188.5
[200]	valid_0's rmse: 77145.3
[300]	valid_0's rmse: 77132.4
[400]	valid_0's rmse: 77127.6
[500]	valid_0's rmse: 77123
[600]	valid_0's rmse: 77118
Early stopping, best iteration is:
[637]	valid_0's rmse: 77116.5


D:\Martin\Anaconda3\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


TypeError: Expected np.float32 or np.float64, met type(int32)

In [11]:
(gbm_pred_test<0).sum()

NameError: name 'gbm_pred_test' is not defined

In [20]:
gbm_model.save_model('modelos/lightgbm_all_installs')

In [15]:
np.sqrt(mean_squared_error(train_Y, gbm_pred_train))

NameError: name 'gbm_pred_train' is not defined

In [16]:
np.sqrt(mean_squared_error(test_Y, gbm_pred_test))

NameError: name 'gbm_pred_test' is not defined

In [14]:
parameters = {
    'num_leaves': [130, 150, 170],
    'max_bin': [150, 200, 250],
    'learning_rate': [0.005, 0.01, 0.2],
    'verbose': [0],
    'num_iterations': [100],
    'max_depth': [12, 15, 20],
    'bagging_fraction': [0.4, 0.5, 0.6]
} verbose=0, num_leaves=150, num_iterations=100, max_depth=15, max_bin=200, learning_rate=0.2, bagging_fraction=0.5

cv_gbm = RandomizedSearchCV(lgb.LGBMModel(objective = 'regression',
            boosting_type='gbdt', num_boost_round=100, verbose_eval=150), param_distributions=parameters, cv=5, scoring='r2', n_iter=20, verbose=5)

In [15]:
cv_gbm.fit(train_X, train_Y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] verbose=0, num_leaves=120, num_iterations=100, max_depth=20, max_bin=100, learning_rate=0.01, bagging_fraction=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
D:\Martin\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[CV]  verbose=0, num_leaves=120, num_iterations=100, max_depth=20, max_bin=100, learning_rate=0.01, bagging_fraction=1, score=0.17151472942623847, total=  38.1s
[CV] verbose=0, num_leaves=120, num_iterations=100, max_depth=20, max_bin=100, learning_rate=0.01, bagging_fraction=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   50.3s remaining:    0.0s
D:\Martin\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[CV]  verbose=0, num_leaves=120, num_iterations=100, max_depth=20, max_bin=100, learning_rate=0.01, bagging_fraction=1, score=0.17095857110959578, total=  46.4s
[CV] verbose=0, num_leaves=120, num_iterations=100, max_depth=20, max_bin=100, learning_rate=0.01, bagging_fraction=1 


...

[CV]  verbose=0, num_leaves=10, num_iterations=100, max_depth=20, max_bin=300, learning_rate=0.001, bagging_fraction=0.5, score=0.032296310696624975, total=  36.7s
[CV] verbose=0, num_leaves=10, num_iterations=100, max_depth=20, max_bin=300, learning_rate=0.001, bagging_fraction=0.5 


D:\Martin\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[CV]  verbose=0, num_leaves=10, num_iterations=100, max_depth=20, max_bin=300, learning_rate=0.001, bagging_fraction=0.5, score=0.03252596264450103, total=  35.6s
[CV] verbose=0, num_leaves=10, num_iterations=100, max_depth=20, max_bin=300, learning_rate=0.001, bagging_fraction=0.5 


D:\Martin\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


MemoryError: 

In [7]:
with open('params.txt', mode='w') as params:
    params.write(str(cv_gbm.best_params_))

In [33]:
np.sqrt(mean_squared_error(train_Y, cv_gbm.predict(train_X)))

89692.2141998931

In [34]:
np.sqrt(mean_squared_error(test_Y, cv_gbm.predict(test_X)))

94441.4408890201

In [72]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'boosting_type': 'gbdt',
                'objective': 'regression',
                'metric': {'l2_root'},
                'num_leaves': 70,
                'max_bin': 400,
                'learning_rate': 0.001,
                'verbose': 0}
xg_reg_cv = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1)
clf = GridSearchCV(xg_reg_cv, parameters, scoring='neg_mean_squared_error', verbose=10)
clf.fit(train_X, train_Y)

**Promedio ponderado**

In [121]:
len_test_X = test_X.shape[0]
gbm_pred_test = gbm_pred_test.reshape(len_test_X, )
xg_pred_test = xg_pred_test.reshape(len_test_X, )
nn_pred_test = nn_pred_test.reshape(len_test_X, )
test_Y = test_Y

In [122]:
np.sqrt(mean_squared_error(test_Y, (gbm_pred_test*0.7 + xg_pred_test*0.1 + nn_pred_test*0.2)))

75282.06926553165

**Tuning XGBoost**

In [ ]:
space  = [Integer(1, 5, name='num_leaves'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(1, n_features, name='max_features'),
          Integer(2, 100, name='min_samples_split'),
          Integer(1, 100, name='min_samples_leaf')]

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set scikit-learn
# estimator parameters
@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=5, n_jobs=-1,
                                    scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)

In [30]:
xgtrain = xgb.DMatrix(train_X, label=train_Y)
xgb_param = xg_reg.get_xgb_params()

cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=40, nfold=4, metrics=['mse'],
     early_stopping_rounds=10, stratified=True, seed=1301)

D:\Program Files\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


XGBoostError: b'[20:35:32] src/metric/metric.cc:21: Unknown metric function mse'

In [28]:
cvresult.

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,100281.798828,25613.922274,145665.577149,61941.546968
1,89032.490235,20409.584995,138748.616211,58368.299514
2,80205.820313,16045.716553,134218.974609,53522.321695
3,72801.069336,12425.053146,131373.916992,48979.007946
4,66812.198242,9596.972955,129332.792969,44736.917297
5,61750.365235,7023.738447,127920.806641,41054.549212
6,57445.105469,4770.600720,126959.982422,38009.871599
7,54163.695313,3207.206254,126242.810547,35500.045521
8,51293.398438,1889.452937,125791.900390,33422.193878
9,48998.215821,827.168210,125467.099609,31758.300831


**Tuning**

In [72]:
from sklearn.model_selection import GridSearchCV

In [123]:
parameters = {'learning_rate':[0.18, 0.2, 0.22], 'max_depth':[11,12], 'gamma':[0], 'n_estimators':[150]}
xg_reg_cv = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1)
clf = GridSearchCV(xg_reg_cv, parameters, scoring='neg_mean_squared_error', verbose=10)
clf.fit(train_X, train_Y)

In [126]:
np.sqrt(mean_squared_error(train_Y, clf.predict(train_X)))/max_seconds

0.09414913459614753

In [127]:
np.sqrt(mean_squared_error(test_Y, clf.predict(test_X)))/max_seconds

0.2620828689044466

In [128]:
clf.best_params_

{'gamma': 0, 'learning_rate': 0.18, 'max_depth': 11, 'n_estimators': 150}